In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import numpy as np
import os
from tqdm.notebook import tqdm

## This Notebook will helps us to create Input Data for each place
**Let's start by visualizing all places we've got**

**Then isolate those places and add the recent column to it LOYALTY**

**The last step will be about defining the shape of our input & output and store it in a folder**

In [3]:
file_lenght=-len(os.path.abspath(".").split("\\")[-1])
csv_path =  os.path.abspath(".")[:file_lenght]+"Data\\"

types={       
    "PARTNERCATEGORY_CODE" : "category" 
    ,"PARTNERCATEGORYTYPE_CODE" : "category" 
    ,"VENDOR" : "category"           
    ,"SECTOR CODE" : "category"         
    ,"SECTOR NAME" : "category"       
    ,"REGION CODE" : "category"        
    ,"REGION NAME" : "category"       
    ,"PRODUCT_CODE" : "category"      
    ,"PRODUCT_NAME" : "category"       
    ,"PRODUCTPAGE_CODE" : "category"    
    ,"UNITPRICE" : "float16"
    ,"SOLD QUANTITY" : "float16"  
    ,"TURNOVER" : "float64"
    ,"DISCOUNT" : "float32" 
    ,"HOLIDAY" : "category"    
    ,"SCOLAR_HOLIDAY" : "category"     
    ,"ISLAM_EVENT" : "string"    
    ,"TEMPERATURE" : "float16"
    ,"HUMIDITY" : "float16"
} # reduce memory consumption
myData = pd.read_csv(csv_path+"Updated_Data_3.csv",header=0,engine='c',index_col=[0],dtype=types)
myData.head(1)

KeyboardInterrupt: 

In [44]:
days= ["sam","lun","dim","mar","mer","jeu","ven","lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"]
newCol=[]
for st in tqdm(myData["ITINERARY_NAME"].tolist()):
    
    st=''.join([i for i in st if not i.isdigit()])
    step1 = st.split(",")
    for i in range(len(step1)):
        step1=step1[:i]+step1[i].split("/")+step1[i+1:]
        
    pre_name=""
    for elm in step1:
        if not elm.lower().replace(' ', '') in days:
            pre_name+=" "+elm
    
    step2 = pre_name.split(" ")
    name=""
    for elm in step2:
        if not (elm.lower() in days):
            name+=" "+elm
    newCol.append(name)

In [45]:
myData["NEW_SECTOR_IT"]=newCol

In [46]:
myData.to_csv(csv_path+"Updated_Data_3.csv")

**now that the sector's name are here, let's see them** 

In [49]:
myData["NEW_SECTOR_IT"].unique()

array(['   Centre ville ', '   Boukadir Sud', '   Tenes ', '   Chorfa',
       '   Sendjas Karimia', '   Zeboudja', '   Abou hassane',
       '   Oued foda ', '   Ain Merane Taougrit', '   Oued Sly Massina',
       '   Centre ville  Moussalaha', '   Chettia', '   Sidi Akacha',
       '   Sobha Boukadir Nord', '  Grossistes', '   Centre vill',
       '   Bellil', "   B.Benchohra K'sar e", '   Aflou ',
       '   Ain madhi   Tadjemou', '   Hassi Delaa',
       '   Gatlat Sidi Saad   E', '   Sidi Makhlouf', '   Lakhneg',
       "   Hassi R'Mel", '   Ain Beida  ', '   Ain Beida ',
       '   Fkirina Ber', '   Hanchir Sou', '   Oum El Bouaghi  ',
       '  Oum El Bouaghi ', '   Oum el Bouaghi ', '   Dhallaa Maskiana ',
       '   Meskiana ', '   Ain Babouche Ksar Sbihi', '  Grossistes ',
       '   Ain Mlila  ', '   Ain Mlila ', '   Ain Mlila   Berteaux',
       '   Ain Kercha ', '   Ain Kercha   Fourchi ', '   Ain Fekroun ',
       '   Ain Fekroun', '   Oum El Bouaghi ', '   Oum El Boua',


**Given a sector name, let's create a csv file for that particulary that contains the loyalty column**

In [55]:
sect_name="Boufarik"
miniData= myData[myData["NEW_SECTOR_IT"].str.contains(sect_name)]
miniData.head(3)

,ITINERARY_NAME,PARTNER_CODE,PARTNERCATEGORY_CODE,PARTNERCATEGORYTYPE_CODE,VENDOR,SECTOR CODE,SECTOR NAME,REGION CODE,REGION NAME,PRODUCT_CODE,...,HOLIDAY,SCOLAR_HOLIDAY,ISLAM_EVENT,TEMPERATURE,HUMIDITY,DAYWEEK,LONGETUDE,LATITUDE,AMAZIGH_EVENT,NEW_SECTOR_IT
2291546,"Sam/Mar, Boufarik 1",09/00016,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000009,...,nouvel an,1,<NA>,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik
2291547,"Sam/Mar, Boufarik 1",09/00541,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000005,...,nouvel an,1,<NA>,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik
2291548,"Sam/Mar, Boufarik 1",09/00541,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000006,...,nouvel an,1,<NA>,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik


In [56]:
y = miniData[["GREGORIAN_DATE","PARTNER_CODE"]].reset_index(drop=True).reset_index().to_numpy()
dict_index = {}
for index,date,clt in y:
    if (date,clt) in dict_index:
        dict_index[(date,clt)].append(index)
    else:
        dict_index[(date,clt)]=[index]

In [57]:
import datetime

clients = {client : -1 for client in miniData["PARTNER_CODE"].drop_duplicates().tolist()}
base = datetime.datetime.strptime("2016-01-01", "%Y-%m-%d")
date_list = [(base + datetime.timedelta(days=x)).strftime("%Y-%m-%d") for x in range(1096)]
result = [0]*len(y)
for date in tqdm(date_list):
    for clt in clients:
        if (date,clt) in dict_index:
            if clients[clt]==-1:
                clients[clt]=0
            for i in dict_index[(date,clt)]:
                result[i]=clients[clt]
            clients[clt]=0
        elif clients[clt]!=-1:
            clients[clt]+=1

In [62]:
miniData["LOYALTY"]=result
# remove NA values
miniData["ISLAM_EVENT"]=miniData["ISLAM_EVENT"].fillna("noEV") 
miniData.head(5)

E:\user\anaconda\envs\Forecasting_sales\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

E:\user\anaconda\envs\Forecasting_sales\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ITINERARY_NAME,PARTNER_CODE,PARTNERCATEGORY_CODE,PARTNERCATEGORYTYPE_CODE,VENDOR,SECTOR CODE,SECTOR NAME,REGION CODE,REGION NAME,PRODUCT_CODE,...,SCOLAR_HOLIDAY,ISLAM_EVENT,TEMPERATURE,HUMIDITY,DAYWEEK,LONGETUDE,LATITUDE,AMAZIGH_EVENT,NEW_SECTOR_IT,LOYALTY
2291546,"Sam/Mar, Boufarik 1",09/00016,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000009,...,1,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik,0
2291547,"Sam/Mar, Boufarik 1",09/00541,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000005,...,1,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik,0
2291548,"Sam/Mar, Boufarik 1",09/00541,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000006,...,1,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik,0
2291549,"Sam/Mar, Boufarik 1",09/00542,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000001,...,1,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik,0
2291550,"Sam/Mar, Boufarik 1",09/00542,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000003,...,1,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Boufarik,0


**Now it's time to create the input/output matrix files**

we decided to create data for each itinerary

In [24]:
miniData = pd.read_csv(csv_path+"Boufarik_Dataset_v2.csv",header=0,engine='c',index_col=[0],dtype=types)
miniData.sort_values(by=["GREGORIAN_DATE"],axis=0,inplace=True)
miniData["ISLAM_EVENT"]=miniData["ISLAM_EVENT"].fillna("noEV") 
miniData.head()

,ITINERARY_NAME,PARTNER_CODE,PARTNERCATEGORY_CODE,PARTNERCATEGORYTYPE_CODE,VENDOR,SECTOR CODE,SECTOR NAME,REGION CODE,REGION NAME,PRODUCT_CODE,...,ISLAM_EVENT,TEMPERATURE,HUMIDITY,DAYWEEK,LONGETUDE,LATITUDE,AMAZIGH_EVENT,MONTH,YEAR,LOYALTY
2291546,"Sam/Mar, Boufarik 1",09/00016,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000009,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
2373102,"Sam/Mar, Boufarik 1",09/00557,GM,GM,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000001,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
2373103,"Sam/Mar, Boufarik 1",09/00557,GM,GM,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000011,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
2373104,"Sam/Mar, Boufarik 1",09/00557,GM,GM,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000019,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
2373105,"Sam/Mar, Boufarik 1",09/00558,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000004,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0


In [29]:
iti_name="Sam/Mar"
iti_data=miniData[miniData["ITINERARY_NAME"].str.contains(iti_name) & miniData["DAYWEEK"]==0].reset_index(drop=True)
iti_data

,ITINERARY_NAME,PARTNER_CODE,PARTNERCATEGORY_CODE,PARTNERCATEGORYTYPE_CODE,VENDOR,SECTOR CODE,SECTOR NAME,REGION CODE,REGION NAME,PRODUCT_CODE,...,ISLAM_EVENT,TEMPERATURE,HUMIDITY,DAYWEEK,LONGETUDE,LATITUDE,AMAZIGH_EVENT,MONTH,YEAR,LOYALTY
0,"Mar/Dim, Boufarik 4",09/00696,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000001,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
1,"Mar/Dim, Boufarik 4",09/00696,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000016,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
2,"Mar/Dim, Boufarik 4",09/00696,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000004,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
3,"Mar/Dim, Boufarik 4",09/00696,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000006,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
4,"Mar/Dim, Boufarik 4",09/00696,M,M,0903R,1605,MD CHERIF SAMIR,16,ALGER,PF-000012,...,noEV,47.84375,0.830078,5,36.7538,3.0588,NoEv,Janauary,2016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77432,"Lun/Sam, Boufarik 3",16/12731,M,M,R16053,1605,MD CHERIF SAMIR,16,ALGER,PF-000001,...,noEV,60.43750,0.890137,0,36.7538,3.0588,NoEv,December,2018,6
77433,"Lun/Sam, Boufarik 3",16/1610111,S,S,R16053,1605,MD CHERIF SAMIR,16,ALGER,PF-000001,...,noEV,60.43750,0.890137,0,36.7538,3.0588,NoEv,December,2018,4
77434,"Lun/Sam, Boufarik 3",16/1610111,S,S,R16053,1605,MD CHERIF SAMIR,16,ALGER,PF-000005,...,noEV,60.43750,0.890137,0,36.7538,3.0588,NoEv,December,2018,4
77435,"Lun/Sam, Boufarik 3",09/00662,M,M,R16053,1605,MD CHERIF SAMIR,16,ALGER,PF-000004,...,noEV,60.43750,0.890137,0,36.7538,3.0588,NoEv,December,2018,4


In [30]:
data = pd.DataFrame(iti_data["GREGORIAN_DATE"].unique())
date = iti_data["GREGORIAN_DATE"].unique()

In [31]:
def Client_Count(Cat,date):
    return iti_data[iti_data["PARTNERCATEGORY_CODE"].str.match(Cat) & iti_data["GREGORIAN_DATE"].str.match(date)].count()["VENDOR"]

Vect_Client=np.vectorize(Client_Count)
Vect_Client("M",data.to_numpy())


array([[  5],
       [ 97],
       [112],
       [119],
       [106],
       [120],
       [122],
       [ 90],
       [101],
       [ 99],
       [ 70],
       [ 95],
       [106],
       [ 96],
       [ 90],
       [ 77],
       [ 88],
       [122],
       [ 81],
       [ 11],
       [103],
       [115],
       [ 85],
       [100],
       [120],
       [ 98],
       [108],
       [103],
       [103],
       [ 85],
       [ 99],
       [ 87],
       [ 98],
       [ 88],
       [ 88],
       [ 62],
       [ 90],
       [ 97],
       [ 88],
       [106],
       [ 98],
       [125],
       [ 88],
       [ 99],
       [  0],
       [ 46],
       [ 44],
       [ 45],
       [ 71],
       [ 30],
       [ 44],
       [ 44],
       [ 71],
       [ 55],
       [ 71],
       [ 32],
       [ 38],
       [ 39],
       [ 81],
       [ 69],
       [ 68],
       [ 44],
       [ 50],
       [ 50],
       [ 70],
       [ 60],
       [ 64],
       [ 45],
       [ 49],
       [ 54],
       [ 55],
      

In [32]:
cat_class = ["G","M","DG","FF","K","GM","0","CAF","S","PL","P"]
for cat in tqdm(cat_class):
    data[cat]=Vect_Client(cat,date)
    print(f"{cat} Done")

G Done


KeyboardInterrupt: 

In [ ]:
amazigh_dict = {miniData["AMAZIGH_EVENT"].unique()[i] : i for i in range(len(miniData["AMAZIGH_EVENT"].unique()))}
print(amazigh_dict)
islam_dict = {miniData["ISLAM_EVENT"].unique()[i] : i for i in range(len(miniData["ISLAM_EVENT"].unique()))}
print(islam_dict)
HOLIDAY_dict = {miniData["HOLIDAY"].unique()[i] : i for i in range(len(miniData["HOLIDAY"].unique()))}
print(HOLIDAY_dict)

In [ ]:
def amazigh(day):
    return amazigh_dict[iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["AMAZIGH_EVENT"].to_numpy()[0]]
vect_amazigh = np.vectorize(amazigh)
data["Amazigh"]=vect_amazigh(date)
print("Amazigh Done")

def islam(day):
    return islam_dict[iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["ISLAM_EVENT"].to_numpy()[0]]
vect_islam = np.vectorize(islam)
data["Islam"]=vect_islam(date)
print("Islam Done")

def holiday(day):
    return HOLIDAY_dict[iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["HOLIDAY"].to_numpy()[0]]
vect_holiday = np.vectorize(holiday)
data["Holiday"]=vect_holiday(date)
print("Holiday Done")

def scholar(day):
    return iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["SCOLAR_HOLIDAY"].to_numpy()[0]
vect_scholar = np.vectorize(scholar)
data["Scholar"]=vect_scholar(date)
print("Scholar Done")

def semaine(day):
    return iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["DAYWEEK"].to_numpy()[0]
vect_semaine = np.vectorize(semaine)
data["Semaine"]=vect_semaine(date)
print("semaine Done")

def humidity(day):
    return iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["HUMIDITY"].to_numpy()[0]
vect_humidity = np.vectorize(humidity)
data["Humidity"]=vect_humidity(date)
print("Humidity Done")

def temperature(day):
    return iti_data[iti_data["GREGORIAN_DATE"].str.match(day)]["TEMPERATURE"].to_numpy()[0]
vect_temperature = np.vectorize(temperature)
data["Temperature"]=vect_temperature(date)
print("Temperature Done")

In [ ]:
products=miniData["PRODUCT_NAME"].unique()

In [ ]:
def Product_Sum(prod,date):
    return iti_data[iti_data["PRODUCT_NAME"].str.match(prod) & iti_data["GREGORIAN_DATE"].str.match(date)].sum()["SOLD QUANTITY"]

Vect_Prod=np.vectorize(Product_Sum)

for prod in tqdm(products):
    data[prod]=Vect_Prod(prod,date)
    print(f"{prod} Done")

In [ ]:
data.to_csv("Input_output_bfrk_1Lun.csv")

In [22]:
data

,0,G,M,DG,FF,K,GM,0,CAF,S,...,TWIST O-A 1L,TWIST P-A 1L,TWIST O-M 1L,PREPARATION CULINAIRE,CANDY CARAMEL 125 ML,ICE CREAM - CHOCOLAT,DE 1L SANS LACTOSE,TWIST O-A 125ML,TWIST O-F-B 125ML,ETUI 500G
0,2016-01-02,32,97,0,0,0,32,0,0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-09,26,127,0,0,0,26,0,0,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-10,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-16,18,98,0,0,0,18,0,0,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-01-23,31,87,0,0,0,31,0,0,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,2018-12-17,21,79,0,0,3,21,0,0,9,...,6.0,0.0,12.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0
429,2018-12-22,13,66,0,0,0,13,0,0,7,...,0.0,0.0,0.0,36.0,0.0,0.0,24.0,0.0,0.0,0.0
430,2018-12-24,17,77,0,0,0,17,0,0,2,...,0.0,0.0,0.0,45.0,0.0,0.0,0.0,72.0,0.0,0.0
431,2018-12-29,21,72,0,0,0,21,0,0,6,...,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,36.0


In [23]:
print(data.columns)

Index([                      0,                     'G',
                           'M',                    'DG',
                          'FF',                     'K',
                          'GM',                     '0',
                         'CAF',                     'S',
                          'PL',                     'P',
                     'Amazigh',                 'Islam',
                     'Holiday',               'Scholar',
                     'Semaine',              'Humidity',
                 'Temperature',            'CHOCO 20CL',
                       'DE 1L',        'TWIST O-M 20CL',
              'TWIST P-A 20CL',                  'VIVA',
                   'ORANGE 1L',      'TWIST O-F-B 20CL',
                  'SILHOUETTE',              'CHOCO 1L',
                      'ENTIER',           'ORANGE 20CL',
                 'COCKTAIL 1L',         'COCKTAIL 20CL',
              'TWIST O-A 20CL',            'CITRONNADE',
                     'DE 50CL',